In [ ]:
import pandas as pd
import GladUtil as gd

COMP_LIFE = {'흥국생명':'흥국','한화생명':'한화','푸르덴셜':'푸르','카디프생명':'카디프','처브라이프':'처브','오렌지라이프':'오렌지','신한생명':'신한','삼성생명':'삼성',
             '미래에셋':'미래','메트라이프':'메트','라이나생명':'라이나','동양생명':'동양','농협생명':'농협','교보생명':'교보','KDB생명':'kdb','KB생명':'KB','IBK':'IBK',
             'DGB생명':'DGB','DB생명':'DB','ABL생명':'ABL'}
COMP_LONG = {'흥국화재':'흥국','한화손보':'한화','현대해상':'현대','삼성화재':'삼성','농협손보':'농협','DB손보':'DB','KB손보':'KB','메리츠손보':'메리츠',
             '롯데손보':'롯데','MG손보':'MG','AIG손보':'AIG','처브손보':'처브','하나손보':'하나','DAS':'DAS','chubb손보':'처브','메리츠화재':'메리츠'}
DICT_JONG = {'인보장':'보장','재물성':'재물','연금성':'연금','저축성':'저축','재물10년':'재물','연금10년':'연금','일반보장1':'보장','보장성':'보장'}

def sum_rate(row, sq_max):
#     print(row)
    sum_rate = 0
    for i in range(1,sq_max):
        col_nm = f'기간{i}-요율'
        if row[col_nm] !=0:
            c1 = f'기간{i}-시작'
            c2 = f'기간{i}-끝'
            cnt = row[c2] - row[c1] + 1
            sum_rate += row[col_nm] * cnt
    return sum_rate

### IBSS에 입력된 수수료 지급기준율과 엑셀파일간 상호 검증   
비교파일
1. 수수료지급율표.ipynb로 만든 IBSS업로드 파일
2. IBSS 수수료메뉴에서 다운받은 지급율 파일

작업결과로 생성된 파일의 요율X와 요율Y가 다른건은 오류이므로 확인 필요

In [ ]:
path1 = 'D:/temp/수수료표/'
in_file1 = '표준_손보 지급율_upload-0.65.xlsx'
sheet1 = 'Worksheet'

path2 = 'D:/temp/수수료표/'
in_file2= '장기_2021-01 장기_2021-01 손보수수료(상동FC)_202101.xlsx'
sheet2 = 'Worksheet'
보종 = '손보'

df = pd.read_excel(path1+in_file1,sheet_name=sheet1,header=0)
df = df.fillna(0)
if 보종=="생보":
    num_list = [ str(i) for i in range(1,10)]
    df['정산타입'] = [ i if i[len(i)-1] in num_list else i +'1' for i in df['정산타입']]

if 보종 == '생보':
    df['요율합'] = df.apply(lambda x: sum_rate(x,11), axis=1)
else:
    df['요율합'] = df.apply(lambda x: sum_rate(x,9), axis=1)

In [ ]:
df_2 = pd.read_excel(path2+in_file2,sheet_name=sheet2,header=2)
df_2 = df_2.fillna(0)
df_2['요율합'] = df_2.apply(lambda x: sum_rate(x,8), axis=1)
# df_2 = df_2.merge(df[['보험사','정산타입','요율합']],on=['보험사','정산타입'],how='left').fillna(0)
df_2 = df_2.join(df.set_index(['보험사','정산타입'])['요율합'], on=['보험사','정산타입'],rsuffix='_엑셀')

In [ ]:
df_temp = df_2[abs(df_2.요율합 - df_2.요율합_엑셀)>0.5]
outFile = f'd:/temp/수수료표/손보아이리치fc-err.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_temp.to_excel(writer, sheet_name='지급율',index=False)

In [ ]:
df_temp

## 글래드조은 양식을 비교하는 스크립트

In [6]:
보종 = '손보'

path1 = 'D:/글로벌백업(180329)/03.수수료&시상/수수료규정/수수료개정_2021/지점별/'
in_file1 = '엠피조은_생손보수수료_202101.xlsx'
sheet1 = '엠피조은- 손보 '

path2 = 'D:/temp/수수료표/'
in_file2= '장기_2021-01 손보수수료(KJ_엠피조은)_202101.xlsx'
sheet2 = 'Worksheet'

df = pd.read_excel(path1+in_file1,sheet_name=sheet1,header=4)
cols= list(df.columns)
cols[:8] = ['회사','보종','상품','납기','환산','fi1','fil2','초회']
df.columns = cols

df['회사'].fillna(method='ffill',inplace=True)           
df['회사'] = df['회사'].str.replace(' ','')
df['보종'] = df.보종.replace(0,None)
df['보종'].fillna(method='ffill',inplace=True)

df['회사'] = df['회사'].apply(lambda x: gd.convertByVal(x,COMP_LONG))
df['보종'] = df['보종'].apply(lambda x: gd.convertByVal(x,DICT_JONG))
df = df.fillna(0)
df['초회'] = df['초회'] * 100
df['지급계'] = df['지급계'] * 100

In [7]:
# 파일2
df_2 = pd.read_excel(path2+in_file2,sheet_name=sheet2,header=2)
df_2 = df_2.fillna(0)
df_2['요율합'] = df_2.apply(lambda x: sum_rate(x,8), axis=1)
df_2 = df_2.join(df.set_index(['회사','보종'])[['초회','지급계']], on=['보험사','정산타입']).fillna(0)

In [8]:

df_temp = df_2[(abs(df_2.초회 - df_2['기간1-요율'])>0.5) | (abs(df_2.요율합 - df_2.지급계)>0.5)]
outFile = f'd:/temp/수수료표/err.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_temp.to_excel(writer, sheet_name='지급율',index=False)